In [ ]:
# Scrape images from url into correct file structure to be fed into a CNN

import pandas as pd
import numpy as np
import csv
from bs4 import BeautifulSoup as soup
from matplotlib.image import imread
import matplotlib.pyplot as plt
import urllib.request
import os
import seaborn as sns

def scrape_image_data(dataframe, list_of_tag_names, batch_per_tag, train_size, file_path):

    train_test_split_counter = np.ceil(batch_per_tag*train_size)
    
    for tag in list_of_tag_names:
        copy_df = df.copy()
        copy_df[tag] = df["tags"].str.contains(tag)
        df_title = f"{tag}_df"
        df_title = copy_df[copy_df[tag]==True]
        # return the number of batches specified. If there are fewer rows in the data frame than
        # batch size, inform the user.
        if len(df_title)>batch_per_tag:
            df_title = df_title.iloc[:batch_per_tag]
        else:
            print(f"There are only " + str(len(df_title)) + " rows that match this tag")
            user_input = input("Type 'Yes' if you would like to use all rows, or 'no' if you would like to cancel the function?")
            if user_input =="Yes":
                pass
            else:
                break
        no_label = df.copy()
        no_label = copy_df[~copy_df[tag]==True]
        if len(no_label)>batch_per_tag:
            no_lable=no_label.iloc[:len(df_title)]
        else:
            print(f"There are only " + str(len(no_label)) + " rows that don't match this tag")
            print(f"Try a different train size.")
            break
        no_lable = no_label.iloc[:len(df_title)]
        labels = [df_title, no_label]
        counter=0
        labels_counter=0
        for image_type in labels:
            if labels_counter==1:
                counter=0
                class_label="no"
                label_edit="not_"
            else:
                class_label="yes"
                label_edit=""
                labels_counter+=1
            list_of_urls = image_type["sample_url"].values.tolist()
            for i, url_title in enumerate(list_of_urls):
                if counter<train_test_split_counter:
                    try:
                        if os.path.isdir(f"{file_path}{tag}/train/{class_label}"):
                            urllib.request.urlretrieve(f"https:{url_title}",f"{file_path}{tag}/train/{class_label}/{label_edit}{tag}{i+1}.png")
                            counter+=1
                        else:
                            os.mkdir(f"{file_path}{tag}")
                            os.mkdir(f"{file_path}{tag}/train")
                            os.mkdir(f"{file_path}{tag}/train/yes")
                            os.mkdir(f"{file_path}{tag}/train/no")
                            urllib.request.urlretrieve(f"https:{url_title}",f"{file_path}{tag}/train/{class_label}/{label_edit}{tag}{i+1}.png")
                            counter+=1
                    except:
                        counter+=1
                        continue
                else:
                    try:
                        if os.path.isdir(f"{file_path}{tag}/test/{class_label}"):
                            urllib.request.urlretrieve(f"https:{url_title}",f"{file_path}{tag}/test/{class_label}/{label_edit}{tag}{i+1}.png")
                            counter+=1
                        else:
                            os.mkdir(f"{file_path}{tag}/test")
                            os.mkdir(f"{file_path}{tag}/test/yes")
                            os.mkdir(f"{file_path}{tag}/test/no")
                            urllib.request.urlretrieve(f"https:{url_title}",f"{file_path}{tag}/test/{class_label}/{label_edit}{tag}{i+1}.png")
                            counter+=1
                    except:
                        counter +=1
                        continue

        del df_title
        del no_label
        
mylist = ["souryuu_asuka_langley"]
file_path = "../Resources/anime_categories/"
scrape_image_data(df,mylist,6700,.75, file_path)
    